In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("Spark DataFrame") \
    .getOrCreate()

24/05/19 21:34:34 WARN Utils: Your hostname, godwolf-2-7 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/05/19 21:34:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/19 21:34:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Bai tap 1: Tao va thao tac voi DataFrame bang JSON

In [24]:
# Đọc dữ liệu từ tệp
df_1 = spark.read.json("/home/godwolf/Downloads/spark-3.5.1-bin-hadoop3/python/TH4/TH4_Attachments/people.json")

In [25]:
# Hiển thị nội dung của DataFrame ra stdout
df_1.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [26]:
# In ra stdout cấu trúc của DataFrame theo dạng cây
df_1.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [27]:
# Chọn duy nhất một cột “name” để hiển thị
df_1.select("name").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



In [28]:
# Hiển thị toàn bộ các cột với tất cả giá trị của cột “age” tăng thêm 1
df_1.select(df_1['name'], df_1['age'] + 1).show()

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     NULL|
|   Andy|       31|
| Justin|       20|
+-------+---------+



In [29]:
# Lọc ra những người có tuổi lớn hơn 21
df_1.filter(df_1['age'] > 21).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [30]:
# Đếm số người theo từng độ tuổi
df_1.groupBy("age").count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|NULL|    1|
|  30|    1|
+----+-----+



# Bai tap 2: Tao va thao tac voi DataFrame tu file CSV

In [31]:
# Tạo DataFrame từ nội dung của file CSV với câu lệnh sau:
df_2 = spark.read.format("csv").option("header", "true").load("/home/godwolf/Downloads/spark-3.5.1-bin-hadoop3/python/TH4/TH4_Attachments/ebay.csv")

In [32]:
# Hiển thị nội dung của DataFrame ra stdout
df_2.show(23)

+----------+-----+--------+--------------+----------+-------+-----+----+----------+
| auctionid|  bid| bidtime|        bidder|bidderrate|openbid|price|item|daystolive|
+----------+-----+--------+--------------+----------+-------+-----+----+----------+
|8213034705|   95|2.927373|      jake7870|         0|     95|117.5|xbox|         3|
|8213034705|  115|2.943484| davidbresler2|         1|     95|117.5|xbox|         3|
|8213034705|  100|2.951285|gladimacowgirl|        58|     95|117.5|xbox|         3|
|8213034705|117.5|2.998947|       daysrus|        10|     95|117.5|xbox|         3|
|8213060420|    2|0.065266|    donnie4814|         5|      1|  120|xbox|         3|
|8213060420|15.25|0.123218|   myreeceyboy|        52|      1|  120|xbox|         3|
|8213060420|    3|0.186539|  parakeet2004|         5|      1|  120|xbox|         3|
|8213060420|   10| 0.18669|  parakeet2004|         5|      1|  120|xbox|         3|
|8213060420|24.99|0.187049|  parakeet2004|         5|      1|  120|xbox|    

In [33]:
# In ra stdout cấu trúc của DataFrame theo dạng cây
df_2.printSchema()

root
 |-- auctionid: string (nullable = true)
 |-- bid: string (nullable = true)
 |-- bidtime: string (nullable = true)
 |-- bidder: string (nullable = true)
 |-- bidderrate: string (nullable = true)
 |-- openbid: string (nullable = true)
 |-- price: string (nullable = true)
 |-- item: string (nullable = true)
 |-- daystolive: string (nullable = true)



In [34]:
# Đếm số cuộc đấu giá đã được tổ chức bằng câu lệnh
df_2.select("auctionid").distinct().count()

627

In [35]:
# Đếm số lượng giá đấu trên mỗi mặt hàng
df_2.groupBy("auctionid", "item").count().show()

+----------+-------+-----+
| auctionid|   item|count|
+----------+-------+-----+
|1645883276|cartier|   12|
|1641242797|cartier|    5|
|1642005995|cartier|   18|
|3020060476|   palm|   22|
|1644681469|cartier|   14|
|8212145833|   xbox|   26|
|1638844464|cartier|   16|
|3020573049|   palm|    2|
|3015632013|   palm|   28|
|3024659380|   palm|   26|
|3025373736|   palm|   15|
|8213387659|   xbox|   25|
|1643201832|cartier|    4|
|3014834745|   palm|   21|
|3020257463|   palm|   16|
|3020274575|   palm|   11|
|3019821569|   palm|   27|
|3024707992|   palm|   20|
|8212277835|   xbox|    3|
|3019326870|   palm|   33|
+----------+-------+-----+
only showing top 20 rows



In [36]:
# Lọc và hiển thị tất cả những phiên đấu giá có giá đóng cửa lớn hơn 100
highprice = df_2.filter("price > 100")
highprice.show()

+----------+-----+--------+--------------+----------+-------+-----+----+----------+
| auctionid|  bid| bidtime|        bidder|bidderrate|openbid|price|item|daystolive|
+----------+-----+--------+--------------+----------+-------+-----+----+----------+
|8213034705|   95|2.927373|      jake7870|         0|     95|117.5|xbox|         3|
|8213034705|  115|2.943484| davidbresler2|         1|     95|117.5|xbox|         3|
|8213034705|  100|2.951285|gladimacowgirl|        58|     95|117.5|xbox|         3|
|8213034705|117.5|2.998947|       daysrus|        10|     95|117.5|xbox|         3|
|8213060420|    2|0.065266|    donnie4814|         5|      1|  120|xbox|         3|
|8213060420|15.25|0.123218|   myreeceyboy|        52|      1|  120|xbox|         3|
|8213060420|    3|0.186539|  parakeet2004|         5|      1|  120|xbox|         3|
|8213060420|   10| 0.18669|  parakeet2004|         5|      1|  120|xbox|         3|
|8213060420|24.99|0.187049|  parakeet2004|         5|      1|  120|xbox|    

# Bai tap 3: Thao tac voi DataFrame bang ngon ngu SQL
# data people

In [37]:
# Tạo một khung nhìn tạm (SQL temporary view) từ DataFrame df
df_1.createOrReplaceTempView("people")

In [38]:
# Truy vấn toàn bộ dữ liệu và hiển thị kết quả
sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [39]:
# Chọn duy nhất một cột “name” để hiển thị
sqlDF = spark.sql("SELECT name FROM people")
sqlDF.show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



In [43]:
# Hiển thị toàn bộ các cột với tất cả giá trị của cột “age” tăng thêm 1
sqlDF = spark.sql("SELECT name, age + 1 AS incremented_age FROM people")
sqlDF.show()

+-------+---------------+
|   name|incremented_age|
+-------+---------------+
|Michael|           NULL|
|   Andy|             31|
| Justin|             20|
+-------+---------------+



In [45]:
# Lọc ra những người có tuổi lớn hơn 21
sqlDF = spark.sql("SELECT * FROM people WHERE age > 21")
sqlDF.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [46]:
# Đếm số người theo từng độ tuổi
sqlDF = spark.sql("SELECT count(name),age FROM people GROUP BY age")
sqlDF.show()

+-----------+----+
|count(name)| age|
+-----------+----+
|          1|  19|
|          1|NULL|
|          1|  30|
+-----------+----+



# Bai tap 3: Thao tac voi DataFrame bang ngon ngu SQL
# data eby

In [47]:
df_2.createOrReplaceTempView("ebay")

In [49]:
# Truy vấn toàn bộ dữ liệu và hiển thị kết quả
sqlDF = spark.sql("SELECT * FROM ebay")
sqlDF.show(23)

+----------+-----+--------+--------------+----------+-------+-----+----+----------+
| auctionid|  bid| bidtime|        bidder|bidderrate|openbid|price|item|daystolive|
+----------+-----+--------+--------------+----------+-------+-----+----+----------+
|8213034705|   95|2.927373|      jake7870|         0|     95|117.5|xbox|         3|
|8213034705|  115|2.943484| davidbresler2|         1|     95|117.5|xbox|         3|
|8213034705|  100|2.951285|gladimacowgirl|        58|     95|117.5|xbox|         3|
|8213034705|117.5|2.998947|       daysrus|        10|     95|117.5|xbox|         3|
|8213060420|    2|0.065266|    donnie4814|         5|      1|  120|xbox|         3|
|8213060420|15.25|0.123218|   myreeceyboy|        52|      1|  120|xbox|         3|
|8213060420|    3|0.186539|  parakeet2004|         5|      1|  120|xbox|         3|
|8213060420|   10| 0.18669|  parakeet2004|         5|      1|  120|xbox|         3|
|8213060420|24.99|0.187049|  parakeet2004|         5|      1|  120|xbox|    

In [52]:
# Đếm số cuộc đấu giá đã được tổ chức bằng câu lệnh
sqlDF = spark.sql("SELECT count(distinct auctionid) FROM ebay")
sqlDF.show()

+-------------------------+
|count(DISTINCT auctionid)|
+-------------------------+
|                      627|
+-------------------------+



In [57]:
# Đếm số lượng giá đấu trên mỗi mặt hàng
sqlDF = spark.sql("SELECT auctionid,item, count(price) FROM ebay GROUP BY auctionid,item")
sqlDF.show()

+----------+-------+------------+
| auctionid|   item|count(price)|
+----------+-------+------------+
|1645883276|cartier|          12|
|1641242797|cartier|           5|
|1642005995|cartier|          18|
|3020060476|   palm|          22|
|1644681469|cartier|          14|
|8212145833|   xbox|          26|
|1638844464|cartier|          16|
|3020573049|   palm|           2|
|3015632013|   palm|          28|
|3024659380|   palm|          26|
|3025373736|   palm|          15|
|8213387659|   xbox|          25|
|1643201832|cartier|           4|
|3014834745|   palm|          21|
|3020257463|   palm|          16|
|3020274575|   palm|          11|
|3019821569|   palm|          27|
|3024707992|   palm|          20|
|8212277835|   xbox|           3|
|3019326870|   palm|          33|
+----------+-------+------------+
only showing top 20 rows



In [58]:
# Lọc và hiển thị tất cả những phiên đấu giá có giá đóng cửa lớn hơn 100
sqlDF = spark.sql("SELECT * FROM ebay WHERE price > 100")
sqlDF.show()

+----------+-----+--------+--------------+----------+-------+-----+----+----------+
| auctionid|  bid| bidtime|        bidder|bidderrate|openbid|price|item|daystolive|
+----------+-----+--------+--------------+----------+-------+-----+----+----------+
|8213034705|   95|2.927373|      jake7870|         0|     95|117.5|xbox|         3|
|8213034705|  115|2.943484| davidbresler2|         1|     95|117.5|xbox|         3|
|8213034705|  100|2.951285|gladimacowgirl|        58|     95|117.5|xbox|         3|
|8213034705|117.5|2.998947|       daysrus|        10|     95|117.5|xbox|         3|
|8213060420|    2|0.065266|    donnie4814|         5|      1|  120|xbox|         3|
|8213060420|15.25|0.123218|   myreeceyboy|        52|      1|  120|xbox|         3|
|8213060420|    3|0.186539|  parakeet2004|         5|      1|  120|xbox|         3|
|8213060420|   10| 0.18669|  parakeet2004|         5|      1|  120|xbox|         3|
|8213060420|24.99|0.187049|  parakeet2004|         5|      1|  120|xbox|    